### Seq2seq

In [ ]:
# import pickle
# from collections import defaultdict

In [1]:
import numpy as np

In [2]:
import numpy as np
import csv
# import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

import numpy as np
import csv
# import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils

In [3]:
# load dataset
with open('../data/food_review.csv') as f:
    reader = csv.reader(f)
    l = list(reader)

In [4]:
n_samples =10000

In [5]:
# encoder and decoder
# Appending SOS andEOS to target data (decoder)
en = [l[i][0] for i in range(1, len(l))] [0:n_samples]
de = ['SOS_ '+ l[i][1] + ' _EOS' for i in range(1, len(l))][0:n_samples]
# en = list()
# de = list()
# for i in range(1, len(l)):
#     en.append(l[i][0])
#     de.append('SOS_ ' + l[i][1] + ' _EOS' )

In [6]:
# num_en_words:  19598
# num_de_words:  4578
# max_en_words_per_sample:  787
# max_de_words_per_sample:  37
# max([len(txt.split(' ')) for txt in en])

max_encoder_seq_length = max([len(txt.split(' ')) for txt in en])
max_decoder_seq_length = max([len(txt.split(' ')) for txt in de])
print(max_encoder_seq_length)
print(max_decoder_seq_length)

782
65


In [7]:
# Create word dictionaries :
en_words=set()
for line in en:
    for word in line.split():
        if word not in en_words:
            en_words.add(word)
    
de_words=set()
for line in de:
    for word in line.split():
        if word not in de_words:
            de_words.add(word)
            
# get lengths and sizes :
num_en_words = len(en_words)
num_de_words = len(de_words)

max_en_words_per_sample = max([len(sample.split()) for sample in en])+5
max_de_words_per_sample = max([len(sample.split()) for sample in de])+5

num_en_samples = len(en)
num_de_samples = len(de)

In [8]:
print('num_en_samples: ', num_en_samples)
print('num_de_samples: ', num_de_samples)
print('num_en_words: ',num_en_words)
print('num_de_words: ',num_de_words)
print('max_en_words_per_sample: ', max_en_words_per_sample)
print('max_de_words_per_sample: ', max_de_words_per_sample)

num_en_samples:  10000
num_de_samples:  10000
num_en_words:  19598
num_de_words:  4578
max_en_words_per_sample:  787
max_de_words_per_sample:  37


In [ ]:
# # Get lists of words :
# input_words = sorted(list(en_words))
# target_words = sorted(list(de_words))

# en_token_to_int = dict()
# en_int_to_token = dict()

# de_token_to_int = dict()
# de_int_to_token = dict()

# #Tokenizing the words ( Convert them to numbers ) :
# for i,token in enumerate(input_words):
#     en_token_to_int[token] = i
#     en_int_to_token[i]     = token

# for i,token in enumerate(target_words):
#     de_token_to_int[token] = i
#     de_int_to_token[i]     = token

In [9]:
en_tokenizer = Tokenizer(num_words=max_encoder_seq_length, char_level=False)
en_tokenizer.fit_on_texts(en)

de_tokenizer = Tokenizer(num_words=max_decoder_seq_length, char_level=False)
de_tokenizer.fit_on_texts(de)

In [10]:
source_token = en_tokenizer.texts_to_sequences(en)
target_token = de_tokenizer.texts_to_sequences(de)
# target_in_token = [0] + target_token[:-1]

source_padded = pad_sequences(source_token, maxlen=max_encoder_seq_length, padding = "post")
target_padded = pad_sequences(target_token, maxlen=max_decoder_seq_length, padding = "post")
print(len(source_padded[0]))
print(len(target_padded[0]))


782
65


In [11]:
# running time calculation
import timeit
start = timeit.default_timer()

# Number of unique input tokens: 84
# Number of unique output tokens: 58
# num_encoder_tokens = 50
num_decoder_tokens = num_de_words #211
n_samples = num_en_samples #100
# prepare data for the LSTM

X1, X2, y = list(), list(), list()
for i in range(n_samples):
# generate source sequence
#     source = source_padded[i] 
#     print('source',source)
    # define target sequence
    target = target_padded[i] 
#     print(target.shape)
#     print(type(target))
#     print('target',target)
    # create padded input target sequence
    target_in = np.insert(target[:-1],0,0)
#     print(target_in.shape)
#     print(type(target_in))
#     print('target_in', target_in)
    # encode
#     src_encoded = utils.to_categorical(source, num_classes=num_encoder_tokens)
    tar_encoded = utils.to_categorical(target, num_classes=num_decoder_tokens)
#     tar2_encoded = utils.to_categorical(target_in, num_classes=num_decoder_tokens)
#     print(src_encoded.shape)
#     print(type(src_encoded))
#     print('src_encoded',src_encoded)
    # store
#     X1.append(src_encoded)
    X2.append(target_in) #tar2_encoded
    y.append(tar_encoded)
#     print(array(X1))
    
# X1 = array(X1)
# X2 = array(X2)
# y = array(y)
encoder_input_data = source_padded #np.array(X1)
decoder_input_data = np.array(X2)
decoder_target_data = np.array(y)

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

# # decode a one hot encoded string
# def one_hot_decode(encoded_seq):
#     return [argmax(vector) for vector in encoded_seq]
 

# print(X1.shape, X2.shape, y.shape)
# print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

# running time check
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

(10000, 782)
(10000, 65)
(10000, 65, 4578)
Time: 2.91 s


In [ ]:
# # read file
# lines = pd.read_csv('food_review.csv')
# # remane col
# lines = lines.rename(columns={'text': 'en', 'summary': 'de'})

In [ ]:
# Appending SOS andEOS to target data

# for i in range(lines.shape[0]):
#     x = lines.loc[i,'de']
#     lines.loc[i,'de']= 'SOS_ '+ x + ' _EOS'

# print(lines.head())

### Embedding from here

In [13]:
# Defining some constants: 
vec_len       = 50#300   # Length of the vector that we willl get from the embedding layer
latent_dim    = 32#1024  # Hidden layers dimension 
dropout_rate  = 0.2   # Rate of the dropout layers
batch_size    = 32#64    # Batch size
epochs        = 1#30    # Number of epochs

###
# num_en_words = 1427
# num_de_words = 211
num_en_words = len(en_words)
num_de_words = len(de_words)
###

# Define an input sequence and process it.
# Input layer of the encoder :
encoder_input = Input(shape=(None,))

# Hidden layers of the encoder :
encoder_embedding = Embedding(input_dim = num_en_words, output_dim = vec_len)(encoder_input)
encoder_dropout   = (TimeDistributed(Dropout(rate = dropout_rate)))(encoder_embedding)
encoder_LSTM      = CuDNNLSTM(latent_dim, return_sequences=True)(encoder_dropout)

# Output layer of the encoder :
encoder_LSTM2_layer = CuDNNLSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM2_layer(encoder_LSTM)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [14]:
# Set up the decoder, using `encoder_states` as initial state.
# Input layer of the decoder :
decoder_input = Input(shape=(None,))

# Hidden layers of the decoder :
decoder_embedding_layer = Embedding(input_dim = num_de_words, output_dim = vec_len)
decoder_embedding = decoder_embedding_layer(decoder_input)

decoder_dropout_layer = (TimeDistributed(Dropout(rate = dropout_rate)))
decoder_dropout = decoder_dropout_layer(decoder_embedding)

decoder_LSTM_layer = CuDNNLSTM(latent_dim, return_sequences=True)
decoder_LSTM = decoder_LSTM_layer(decoder_dropout, initial_state = encoder_states)

decoder_LSTM_2_layer = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
decoder_LSTM_2,_,_ = decoder_LSTM_2_layer(decoder_LSTM)

# Output layer of the decoder :
decoder_dense = Dense(num_de_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_LSTM_2)


In [15]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_input, decoder_input], decoder_outputs)

model.summary()

# Define a checkpoint callback :
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]



num_train_samples = 100 #9000
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data,
               decoder_input_data],
               decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.08,
          callbacks = callbacks_list)

# model.fit([encoder_input_data[:num_train_samples,:],
#                decoder_input_data[:num_train_samples,:]],
#                decoder_target_data[:num_train_samples,:,:],
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.08,
#           callbacks = callbacks_list)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     979900      input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 50)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
embedding_

In [ ]:
# # Use sklearn utility to convert label strings to numbered index
# encoder = LabelEncoder()
# encoder.fit(train_label)
# y_train = encoder.transform(train_label)
# y_test = encoder.transform(test_label)

# # Converts the labels to a one-hot representation
# num_classes = np.max(y_train) + 1
# y_train = utils.to_categorical(y_train, num_classes)
# y_test = utils.to_categorical(y_test, num_classes)